# Text-Based Point Search

In this notebook we're going to learn how we can easily add a search bar to Jupyter Scatter. To demonstrate this, let's look at cities across the world using the [GeoNames](https://www.geonames.org/about.html) dataset. The search bar will allow us to find and select cities by name.

In [ ]:
# If you run this notebook in Google Colab, you need to manually install the following packages.
# !pip install --quiet jupyter-scatter

In [ ]:
!curl -L -C - -o data/cities.pq https://storage.googleapis.com/flekschas/jupyter-scatter-tutorial/cities.pq

In [ ]:
import pandas as pd
cities = pd.read_parquet('data/cities.pq')
cities.head(3)

The key ingredients here are:
1. `ipywidgets.Text` to allow us to enter a city name
2. Pandas DataFrame's `query()` method to find cities by name
3. `scatter.selection()` to select cities
4. `scatter.zoom()` to zoom to selected points

Finally, all we have to do is to observe changes to search box and and call the above mentioned functions.

In [ ]:
import ipywidgets
import jscatter

scatter = jscatter.Scatter(data=cities, x='Mercator X', y='Mercator Y')
scatter.color(by='Continent').size(2).axes(False).legend(True).height(640)

searchbox = ipywidgets.Text(
    value='',
    placeholder='Search by city name',
    description='Search:',
    disabled=False   
)

def search_change_handler(change):
    if change['new']:
        city_idxs = cities.query(f'Name == "{change['new']}"').index
        scatter.selection(city_idxs)
        scatter.zoom(to=city_idxs, animation=500, padding=0.5)
    else:
        scatter.selection(None)
        scatter.zoom(to=None, animation=500, padding=0)

searchbox.observe(search_change_handler, names=['value'])

ipywidgets.VBox([searchbox, scatter.show()])

The beauty of this approach is that you search across any column in the data frame. For instance, in the following example we enhanced the search to allow:
- Searching by population via `n:<number>` or `n:<number>-<number>`. E.g., `n:1000000-10000000` will select all cities with a population between 1 to 10 million.
- Searching by continent. E.g., `Australia` will select all cities in Australia

In [ ]:
import ipywidgets
import jscatter

scatter2 = jscatter.Scatter(data=cities, x='Mercator X', y='Mercator Y')
scatter2.color(by='Continent').size(2).axes(False).legend(True).height(640)

searchbox = ipywidgets.Text(
    value='',
    placeholder='Search by city name',
    description='Search:',
    disabled=False   
)

def search_change_handler(change):
    if change['new']:
        if change['new'] in cities.Continent.unique():
            city_idxs = cities.query(f'Continent == "{change['new']}"').index
        elif change['new'].startswith('n:'):
            if '-' in change['new']:
                n1, n2 = change['new'][2:].split('-')
                city_idxs = cities.query(f'Population >= {n1} and Population <= {n2}').index
            else:
                city_idxs = cities.query(f'Population == {change['new'][2:]}').index
        else:
            city_idxs = cities.query(f'Name == "{change['new']}"').index
        scatter2.selection(city_idxs)
        scatter2.zoom(to=city_idxs, animation=500, padding=0.5)
    else:
        scatter2.selection(None)
        scatter2.zoom(to=None, animation=500, padding=0)

searchbox.observe(search_change_handler, names=['value'])

ipywidgets.VBox([searchbox, scatter2.show()])

As usual, you can get at the data records of the selected cities as follows:

In [ ]:
cities.iloc[scatter2.selection()]